In [92]:
from langchain_ollama import ChatOllama 
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional



In [93]:
model = ChatOllama(model = 'llama3.2:3b')

In [123]:
# define the state
class BlogState(TypedDict, total=False):
    
    topic: str
    outline: str
    content: str
    evaluation_score: str

In [124]:
# Create the outline

def create_outline(state: BlogState) -> BlogState:

#   fetch title 
    topic = state['topic']

# Call llm and gen outline

    prompt = f'Generate a detailed outline for a well structured Blog on the topic: {topic}'
    outline = model.invoke(prompt).content

# update state
    state['outline'] = outline
    return state

In [125]:
# Generate the blog 

def create_blog(state: BlogState) -> BlogState:

    topic = state['topic']
    outline = state['outline']

    prompt = f'Generate a good quality and post worthy content on the topic {topic} using the following outline: {outline}'
    content = model.invoke(prompt).content

    state['content'] = content
    return state 

In [126]:
#  Give a score 

def generate_score(state: BlogState) -> BlogState:

    topic = state['topic']
    outline = state['outline']
    content = state['content']

    prompt = f'You are a highly professional Evaluator of Blogs. You have to just give a number as an output for this and nothing else:  Based on this : {outline}, on the topic: {topic} rate (between the scale of 0 to 1, 1 being the best and 0 being the worst) the following content generated for the blog : {content}'
    evaluation_score = model.invoke(prompt).content
    state['evaluation_score']= evaluation_score

    return state

In [127]:
# Create the graph
graph = StateGraph(BlogState)

# add nodes 
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('generate_score', generate_score)

# add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'generate_score')
graph.add_edge('generate_score', END)

# compile 
workflow = graph.compile()


In [128]:
initial_state = {'topic': 'Rise of AI in India'}
final_state = workflow.invoke(initial_state)

print(final_state)

{'topic': 'Rise of AI in India', 'outline': 'Here\'s a detailed outline for a well-structured blog on "The Rise of AI in India":\n\n**I. Introduction**\n\n* Brief overview of Artificial Intelligence (AI) and its growing importance globally\n* Significance of AI in India\'s economy, industry, and society\n* Thesis statement: The rise of AI in India is transforming the country\'s industries, improving quality of life, and presenting opportunities for growth and development.\n\n**II. Historical Background**\n\n* Overview of India\'s journey with AI from its early beginnings to the present day\n* Discussion of key milestones, such as:\n\t+ India\'s first AI lab established in 2015\n\t+ Government initiatives to promote AI adoption, e.g., National Programme on Artificial Intelligence (NPAI)\n\t+ Indian companies\' involvement in AI startups and research projects\n\n**III. AI Adoption in Indian Industries**\n\n* Analysis of AI adoption across various industries in India:\n\t+ Healthcare: tel

In [120]:
print(final_state['outline'])

Here is a detailed outline for a well-structured blog on "The Rise of AI in India":

**I. Introduction**

* Brief overview of Artificial Intelligence (AI) and its growing importance globally
* Significance of AI in India's economy, industry, and society
* Thesis statement: The rise of AI in India has the potential to transform various sectors, create new opportunities, and drive growth.

**II. Current State of AI in India**

* Overview of India's current AI landscape, including notable achievements and challenges
* Statistics on AI adoption, investment, and employment in India
* Discussion of government initiatives and policies supporting AI growth in India

**III. Emerging Industries driving AI Adoption in India**

* Agriculture: use of AI for crop monitoring, precision farming, and supply chain management
* Healthcare: application of AI in medical diagnosis, personalized medicine, and healthcare management
* Education: role of AI in personalized learning, adaptive assessments, and ed

In [121]:
print(final_state['content'])

**The Rise of AI in India: A Transformative Journey Ahead**

Artificial Intelligence (AI) has been revolutionizing the way we live, work, and interact with each other for over two decades now. The global AI market is expected to reach $190 billion by 2025, growing at a CAGR of 38% from 2020 to 2025 (MarketsandMarkets). India, with its young population, vast talent pool, and growing economy, is poised to be a significant player in the AI revolution.

The rise of AI in India has far-reaching implications for various sectors, creating new opportunities, driving growth, and transforming industries. From agriculture to education, healthcare, and financial services, AI is being increasingly adopted across various domains. This blog post aims to explore the current state of AI in India, emerging industries driving adoption, startups and entrepreneurs playing a crucial role, challenges and concerns, government initiatives and policies, international collaborations, and the future outlook.

**C

In [122]:
print(final_state['evaluation_score'])

0.0
